In [ ]:
import os, re, json
from typing import List, Dict, Any, Tuple
import chromadb
from chromadb.utils import embedding_functions
import google.generativeai as genai
from difflib import SequenceMatcher
import pandas as pd
from tqdm import tqdm
import re
import ast

# ==========================================================
# 🔧 Configuration
# ==========================================================
DB_PATH = "vector_new_db/recipes"   # your ChromaDB path
MODEL_NAME = "gemini-2.0-flash-lite"
GENAI_API_KEY = os.getenv('GENAI_API_KEY') 

TOP_K_DEFAULT = 3
OUTPUT_DIR = "/content/evaluation_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================================================
chroma_client = chromadb.PersistentClient(path=DB_PATH)
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="BAAI/bge-m3"
)
collection = chroma_client.get_collection("recipes")

# ==========================================================
genai.configure(api_key=GENAI_API_KEY)
model = genai.GenerativeModel(MODEL_NAME)

# ==========================================================
TOKEN_CLEAN_RE = re.compile(r"[^a-zA-Z0-9\s\-]")
STOP_TOKENS = {"fresh","dried","chopped","sliced","minced","ground","crushed",
    "thinly","thick","thickly","small","medium","large","whole","skinless",
    "boneless","lean","extra","optional","to","taste","of","and","or"}

CANONICAL_MAP = {
    "chicken thigh":"chicken","chicken thighs":"chicken","chicken breast":"chicken",
    "beef rib":"beef ribs","beef ribs":"beef ribs","pork belly":"pork belly",
    "pork ribs":"pork ribs","pork":"pork",
    "tofu":"tofu","tofu skin":"yuba","bean curd skin":"yuba","yuba":"yuba",
    "garlic":"garlic","ginger":"ginger","chili oil":"chili oil",
    "soy sauce":"soy sauce","doubanjiang":"doubanjiang"
}
CONFLICTS = {("tofu","yuba"),("yuba","tofu")}

def _normalize(s:str)->str:
    s = s.lower().strip()
    s = TOKEN_CLEAN_RE.sub(" ", s)
    return re.sub(r"\s+"," ",s).strip()

def _canonicalize(s:str)->str:
    s_norm=_normalize(s)
    for k in sorted(CANONICAL_MAP,key=len,reverse=True):
        if re.search(rf"(?<![a-z0-9]){re.escape(k)}(?![a-z0-9])",s_norm):
            return CANONICAL_MAP[k]
    return s_norm

def _is_conflict(a,b): return (a,b) in CONFLICTS
def _similar(a,b): return SequenceMatcher(None,a,b).ratio()

def match_ing(a,b,th=0.88):
    a,b=_canonicalize(a),_canonicalize(b)
    if _is_conflict(a,b): return False
    if a==b: return True
    if re.search(rf"(?<![a-z0-9]){re.escape(a)}(?![a-z0-9])",b) or re.search(rf"(?<![a-z0-9]){re.escape(b)}(?![a-z0-9])",a):
        return True
    return _similar(a,b)>=th

# ==========================================================
class RecipeModelEvaluator:
    def __init__(self,collection,embedder,model,top_k:int=3):
        self.collection=collection
        self.embedder=embedder
        self.model=model
        self.top_k=top_k

    def _retrieve_recipes(self,query:str)->List[Dict[str,Any]]:
        emb=self.embedder([query])[0]
        res=self.collection.query(query_embeddings=[emb],n_results=self.top_k)
        docs=res.get("documents",[[]])[0]
        metas=res.get("metadatas",[[]])[0]
        out=[]
        for i in range(min(len(docs),self.top_k)):
            m=metas[i]
            out.append({
                "name":m.get("Recipes_name",""),
                "flavor":m.get("Flavor",""),
                "difficulty":m.get("Difficulty",""),
                "time":m.get("Estimated Cooking Time",""),
                "doc":docs[i]
            })
        return out

    def _build_prompt(self,query:str,cands:List[Dict[str,Any]])->str:
        blocks=[]
        for i,c in enumerate(cands,1):
            blocks.append(f"[Candidate {i}]\nName:{c['name']}\nFlavor:{c['flavor']}\nContent:\n{c['doc']}")
        schema={
            "recipe_name":"string",
            "chosen_from":"string",
            "ingredients":["string","..."],
            "steps":["string","..."],
            "reasoning":"string"
        }
        return f"""
User request:
{query}

Candidates:
{chr(10).join(blocks)}

Output exactly ONE JSON object following this schema:
{json.dumps(schema,indent=2)}
"""

    def generate_recipe(self,prompt:str)->str:
        cands=self._retrieve_recipes(prompt)
        print(f"📦 Retrieved {len(cands)} candidates from Chroma")
        llm_prompt=self._build_prompt(prompt,cands)
        try:
            resp=self.model.generate_content(llm_prompt)
            return resp.text
        except Exception as e:
            return json.dumps({"recipe_name":"ERROR","chosen_from":"ERROR","ingredients":[],"steps":[],"reasoning":str(e)})

    def parse_json(self,text:str)->Dict[str,Any]:
        try: return json.loads(text)
        except: pass
        m=re.search(r"(\{.*\})",text,flags=re.DOTALL)
        if m:
            try: return json.loads(m.group(1))
            except: pass
        return {"recipe_name":"NO_MATCH","chosen_from":"NO_MATCH","ingredients":[],"steps":[],"reasoning":""}

    def calc_recall(self,user_ings:List[str],gen_ings:List[str])->Dict[str,Any]:
        u=[_normalize(x) for x in user_ings]
        g=[_normalize(x) for x in gen_ings]
        match=[]
        mg=set()
        for ui in u:
            for j,gi in enumerate(g):
                if j in mg: continue
                if match_ing(ui,gi):
                    match.append((ui,gi)); mg.add(j); break
        r=len(match)/len(u) if u else 0
        p=len(match)/len(g) if g else 0
        f=2*r*p/(r+p) if (r+p)>0 else 0
        return {"matched":len(match),"recall":r,"precision":p,"f1":f,"pairs":match}

# ==========================================================
model_eval = RecipeModelEvaluator(collection,embedder,model)

df = pd.read_csv("recipes_prompt_label.csv")
df = df.iloc[0:600]  # test on first 600 samples
BATCH_SIZE = 50
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

all_results = []  # store all metrics here

for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Evaluating recipes"):
    batch = df.iloc[i:i + BATCH_SIZE]
    docs = batch["prompt"].tolist()
    ids = batch["ingredients/label"].tolist()

    for j, (prompt, gold_ings_raw) in enumerate(zip(docs, ids)):
        print("\n" + "=" * 80)
        print(f"🧩 Sample {i + j + 1}/{len(df)}")

        print("\n🔍 Prompt:", prompt)
        print("🎯 Ground Truth (raw):", gold_ings_raw)

        if isinstance(gold_ings_raw, str):
            try:
                gold_ings = ast.literal_eval(gold_ings_raw)
                if not isinstance(gold_ings, list):
                    gold_ings = [gold_ings]
            except Exception:
                gold_ings = [x.strip() for x in gold_ings_raw.split(",") if x.strip()]
        else:
            gold_ings = list(gold_ings_raw) if not isinstance(gold_ings_raw, list) else gold_ings_raw

        print("✅ Parsed Ground Truth (list):", gold_ings)

        raw_out = model_eval.generate_recipe(prompt)
        print("\n💬 Gemini Raw Output:\n", raw_out[:800])

        parsed = model_eval.parse_json(raw_out)
        gen_ings = parsed.get("ingredients", [])
        print("\n🥬 Extracted Ingredients:", gen_ings)

        metrics = model_eval.calc_recall(gold_ings, gen_ings)
        print("\n📊 Metrics:\n", json.dumps(metrics, indent=2, ensure_ascii=False))

        metrics_row = {"id": i + j + 1, "prompt": prompt}
        metrics_row.update(metrics)
        metrics_row["raw_output"] = raw_out
        all_results.append(metrics_row)

results_df = pd.DataFrame(all_results)
results_path = os.path.join(OUTPUT_DIR, "metrics_results.csv")
results_df.to_csv(results_path, index=False, encoding="utf-8-sig")

print(f"\n✅ 所有指标已保存到: {results_path}")

Evaluating recipes:   0%|          | 0/12 [00:00<?, ?it/s]


🧩 Sample 1/600

🔍 Prompt: I have pork belly, garlic, and chili oil. I want to make something spicy and Sichuan-style.
🎯 Ground Truth (raw): pork belly, garlic, chili oil
✅ Parsed Ground Truth (list): ['pork belly', 'garlic', 'chili oil']
📦 Retrieved 3 candidates from Chroma

💬 Gemini Raw Output:
 ```json
{
  "recipe_name": "Chilled Pork Belly Salad",
  "chosen_from": "[Candidate 3]",
  "ingredients": [
    "Pork belly",
    "Onion",
    "Garlic",
    "Chili",
    "Soy sauce",
    "Chili oil",
    "Sichuan oil"
  ],
  "steps": [
    "Boil the pork belly until tender. Let it cool and slice thinly.",
    "Marinate the pork belly slices in a mixture of soy sauce, chili oil, and Sichuan oil. Add thinly sliced onion, minced garlic, and chopped chili.",
    "Chill the marinated pork belly for at least 30 minutes to allow the flavors to meld.",
    "Serve the chilled pork belly salad, garnished with additional chili oil or chopped scallions (optional)."
  ],
  "reasoning": "Candidate 3 is the

Evaluating recipes:   8%|▊         | 1/12 [02:02<22:24, 122.21s/it]


💬 Gemini Raw Output:
 ```json
{
  "recipe_name": "Beef with Broccoli",
  "chosen_from": "[Candidate 2]",
  "ingredients": [
    "beef",
    "broccoli",
    "soy sauce"
  ],
  "steps": [
    "Cut the beef into bite-sized pieces.",
    "Cut the broccoli into florets.",
    "Stir-fry the beef in a hot pan until browned.",
    "Add the broccoli and stir-fry until slightly tender.",
    "Pour in soy sauce and stir-fry until the sauce thickens and coats the beef and broccoli.",
    "Serve immediately."
  ],
  "reasoning": "Candidate 2, Broccoli and Mushroom Beef, seems like the closest fit based on the ingredients available and the user's desired dish of beef with broccoli, even if it adds mushrooms.  I've adapted it, removing the mushrooms and oyster sauce (since those weren't listed as ingredients), and the starch 

🥬 Extracted Ingredients: ['beef', 'broccoli', 'soy sauce']

📊 Metrics:
 {
  "matched": 3,
  "recall": 1.0,
  "precision": 1.0,
  "f1": 1.0,
  "pairs": [
    [
      "beef",
  

Evaluating recipes:  17%|█▋        | 2/12 [04:01<20:03, 120.38s/it]

...

Evaluating recipes:  92%|█████████▏| 11/12 [19:52<01:43, 103.82s/it]


💬 Gemini Raw Output:
 ```json
{
  "recipe_name": "Spicy Fried Tofu Cubes",
  "chosen_from": "[Candidate 3]",
  "ingredients": [
    "Tofu",
    "Dried chili",
    "Peppercorn",
    "Garlic",
    "Cumin",
    "Chili powder",
    "Spices",
    "Chili Oil (implied)"
  ],
  "steps": [
    "Press the tofu to remove excess water and cube it.",
    "Heat chili oil in a wok or large pan over medium-high heat.",
    "Add the tofu cubes and fry until golden brown and crispy.",
    "Add dried chili, peppercorn, minced garlic, cumin, chili powder, and other spices to the wok and stir-fry until fragrant.",
    "Stir to coat the tofu with the spice mixture.",
    "Serve immediately."
  ],
  "reasoning": "The user requested a spicy stir-fry using tofu, mushrooms, and chili oil. Candidate 3 best fits this request. While Candid

🥬 Extracted Ingredients: ['Tofu', 'Dried chili', 'Peppercorn', 'Garlic', 'Cumin', 'Chili powder', 'Spices', 'Chili Oil (implied)']

📊 Metrics:
 {
  "matched": 2,
  "recall": 0

Evaluating recipes: 100%|██████████| 12/12 [21:35<00:00, 107.99s/it]


💬 Gemini Raw Output:
 ```json
{
  "recipe_name": "Braised Pork Ribs",
  "chosen_from": "Candidate 2",
  "ingredients": [
    "Pork ribs",
    "Ginger",
    "Garlic",
    "Soy sauces",
    "Sugar",
    "Wine"
  ],
  "steps": [
    "Sear the pork ribs until browned.",
    "Sauté ginger and garlic.",
    "Add soy sauce, sugar, and wine. Bring to a simmer.",
    "Add the ribs to the braising liquid. Braise until tender.",
    "Serve."
  ],
  "reasoning": "Candidate 2 provides a braised pork ribs recipe using the key ingredients: pork ribs, soy sauce, and sugar. The other candidates include ingredients the user didn't mention or have a different flavor profile (Candidate 3) that is outside of the user's explicit request."
}
```

🥬 Extracted Ingredients: ['Pork ribs', 'Ginger', 'Garlic', 'Soy sauces', 'Sugar', 'Wine']

📊 Metrics:
 {
  "matched": 3,
  "recall": 1.0,
  "precision": 0.5,
  "f1": 0.6666666666666666,
  "pairs": [
    [
      "pork ribs",
      "pork ribs"
    ],
    [
      "soy

PermissionError: [Errno 13] Permission denied: 'outputs\\metrics_results.csv'

In [12]:
results_df.to_csv(results_path, index=False, encoding="utf-8-sig")